Theano Tutorial @ LTI, Carnegie Mellon University
=======================

Kazuya Kawakami, clab

# Theano

Theano is a Python library, like numpy, that lets you to define mathematical expressions and run them on GPU.  
In short, Theano is the best prototyping tool!!

**Pros**

- **Python** : Preprocessing, Modeling, Visualization. Numpy like expression.
- **Easy to access GPU**: You don't need to do anything!!
- **Execution speed optimizations**: Theano can use g++ or nvcc to compile your expression graph
- **Symbolic differentiation**: Automatic Differentiation on symbolic expressions
- **Stability optimizations**: Theano recognize numerically unstable expressions and fix them
- **Still growing**: Developer communitiy is active

**Cons**

- **Loop** : Restrictions on how the loop interact with the rest of the graph
- ** Goto/Recursion**: are not supprted

## Contents
0. **Tools**: You don't need to know theano at all !! ( [nolearn](https://github.com/dnouri/nolearn.git), [Pylearn2](http://deeplearning.net/software/pylearn2/), [sklearn-theano](http://sklearn-theano.github.io/auto_examples/plot_mnist_generator.html#example-plot-mnist-generator-py) ).

1. **Overview**: How theano codes look like??
2. **Variables**: Symbolic variable, Shared variable
3. **Function, Computational Graph**: tensor.function, tensor.clone, theano.printing.pp, theano.printing.debugprint
4. **Math**:  Comparison, Condition
5. **Linear Algebra**
6. **Gradient**: theano.gradient.grad, theano.gradient.hessian, theano.gradient.jacobian, update
7. **GPU**: Data type 
8. **Linear Regression**
9. **Multi Layer Perceptron**
10. **Convolution**
11. **Maxpooling**
12. **Scan**
13. **Recurrent Neural Networks**
14. **Tips for debugging**
15. **Links**

## Server

This is a hands-on tutorial, if you sent your public key to Prasanna, you can log in to your server.
The server has 4 GPUs. To avoid all participants uses one gpu, please specify explicitly the gpu you want to use.

In [1]:
import theano.sandbox.cuda
theano.sandbox.cuda.use("gpu0") #gpu1, gpu2, gpu3

Using gpu device 0: GeForce GTX TITAN X


If you are running command line, run your code with the following options.

```bash
THEANO_FLAGS=mode=FAST_RUN,device=gpu0,floatX=float32, python your_code.py
THEANO_FLAGS=mode=FAST_RUN,device=gpu1,floatX=float32, python your_code.py
THEANO_FLAGS=mode=FAST_RUN,device=gpu2,floatX=float32, python your_code.py
THEANO_FLAGS=mode=FAST_RUN,device=gpu3,floatX=float32, python your_code.py
```

Then launch your ipython server by following this [Instruction](http://ipython.org/ipython-doc/1/interactive/public_server.html).

## Tools

There are a lots of tools implemented with Theano.
For example, [nolearn](https://github.com/dnouri/nolearn.git) let you write image classification in 30 lines !! 

More examples and tutorials are in [dl_tutorial](https://github.com/oduerr/dl_tutorial/tree/master/lasagne).

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
from collections import OrderedDict

import numpy
import theano
import theano.tensor as T
from theano.sandbox.rng_mrg import MRG_RandomStreams as RandomStreams

trng = RandomStreams(42)

from sklearn.datasets import fetch_mldata
from sklearn.utils import shuffle
from sklearn.cross_validation import train_test_split
from sklearn.metrics import f1_score

#Random Seed
rng = numpy.random.RandomState(1234)

mnist = fetch_mldata('MNIST original')

# mnist_x is a (n_sample, n_feature=784) matrix
mnist_x, mnist_y = shuffle(mnist.data.astype("float32")/255.0, mnist.target.astype("int32"))

In [ ]:
## Show Randomly picked Examples
def plot_sample(x, axis):
    img = x.reshape(28, 28)
    axis.imshow(img, cmap='gray')
    
fig = plt.figure(figsize=(6, 6))
for i in range(36):
    ax = fig.add_subplot(6, 6, i + 1, xticks=[], yticks=[])
    plot_sample(mnist_x[numpy.random.randint(0,60000)], ax)

## Overview of Theano code

1. Define Symbolic/Shared variables (**Variables**)
2. Construct a computational graph (**Math**)
3. Compile the graph (**Function**)
4. Run!!

In [ ]:
# Linear Regression
import numpy
import theano
import theano.tensor as T

##  Step1. Define Symbolic / Shared Variables
x, t = T.fvector("x"), T.fvector("t") #input

W = theano.shared(rng.uniform(low=-0.08, high=0.08, size=(5, 3)), name="W") #variables that are shared over iteration: weight, bias
b = theano.shared(numpy.zeros(3), name="bias")


## Step2. Define graph
y = T.dot(x, W) + b

cost = T.sum((y - t)**2) #Cost function

gW, gb = T.grad(cost, [W, b]) # Take gradient

updates =  OrderedDict({W: W-0.01*gW, b: b-0.01*gb}) # Set update expression in OrderedDict


## Step3. Compile graph
f = theano.function(inputs=[x, t], outputs=[cost, gW, gb], updates=updates, allow_input_downcast=True)


## Step4. Run!!
for epoch in range(5):
    cost, gW, gb = f([-2., -1., 1., 2., 3.], [.4, .3, .5])
    print("epoch:: %d, cost:: %.3f"%(epoch, cost))

## Variables

In addition to numpy.ndarray, you need to know **Symbolic Variable** and **Shared Variable**.

- **Symbolic Variable** is a symbolic representation of quantities you want to use in functions. (Inputs)
- **Shared Variable** is a variable with **storage** that is shared between functions. (Weights, Data)

In [ ]:
def identity(a):
    return a

a = T.iscalar('a')
f = theano.function([a], a)

print(identity(1))
print(f(1))

In [ ]:
W = theano.shared(numpy.array([0., 1., 2., 3., 4.]).astype("float32"), name="W")

print(W)
print(W.get_value())

In [ ]:
## Symbolic Variables 
a = T.iscalar("a") # integer
b = T.fscalar("b") # float scalar

x = T.fvector("x") # float vector
X = T.fmatrix("X") # float matrix

## Shared Variable, store variables on cpu/gpu memory
W = theano.shared(numpy.array([0., 1., 2., 3., 4.]).astype("float32"), name="W")
bias  = theano.shared(numpy.float32(5), name="bias")

# Get Value from shared variable
print(W.get_value())

## Define symbolic graph
c = a + b
y = T.dot(x, W) + bias

## Print symbolic graph
print(theano.pp(y))

##  Advanced:: You can replace some parts of computation graph with different variable
d = theano.clone(output=c, replace={b: y}) #replace "b" with "y"
print(theano.pp(d))

## Function

Compile symbolic graph into a function

** BE CAREFUL**:: YOU NEED TO USE int32 or float32, only computations with float32 data-type can be accelerated.

In [ ]:
## Compile symbplic graph into callable functions
add = theano.function(inputs=[a, b], outputs=c)
linear = theano.function(inputs=[x], outputs=y)

## Call Functions
print(add(1, 5))
print(linear([0., 0., 0., 0., 1.]).astype("float32"))

##Print function
theano.printing.debugprint(linear)

## Advanced :: You can evaluate symbolic graph without compilation
print(c.eval({
            a : numpy.int32(16), 
            b : numpy.float32(12.10)
        }))

## Math

Most mathmatic operations are similar to numpy. However, comparison operations have special form. [[Basics](http://deeplearning.net/software/theano/library/tensor/basic.html)]

 Condition operation is 
 ```python
 T.switch(condition, if true, if false)
 ```
 
 Comparison is 
 
 ```python
 T.gt(a, b) #Greater Than
 ```

In [ ]:
x = T.fvector("x")

### Basic Math operation & Activation funcsions
exp_x = T.exp(x)
sigmoid_x = T.nnet.sigmoid(x)
tanh_x = T.tanh(x)

### Advanced:: condition and comparison
relu_x = T.switch(T.gt(x, 0), x, 0)

f = theano.function([x], [exp_x, sigmoid_x, tanh_x, relu_x])
f(numpy.array([-2., -1., 1., 2., 3.]).astype("float32"))

## Gradient (Automatic Differentiation)

You can define gradient symbolically. Amazing!!

If you want to use Jacobian or Hessian, use theano.gradient.jacobian,  theano.gradient.hessian.

In [ ]:
# y = x ** 2
x = T.fscalar("x")
y = x ** 2
gy = theano.grad(cost=y, wrt=x) ## 2x

f = theano.function([x], [y, gy]) ## x**2, 2x
print(f(10))

**Updateing your parameters (shared variables)** is the key process, but it's a bit complicated.

Let's start from a toy example.

In [ ]:
a = 0
for i in range(5):
    print(a)
    a += 1

In [ ]:
##Define a function which update t by 1 for each call.

t = theano.shared(numpy.int32(0))
increment = theano.function([], t, updates=OrderedDict({t: t+1}) ) #OrderedDict({before update: after update})
for i in range(5):
    t = increment()
    print(t)

In [ ]:
# Linear Regression
from collections import OrderedDict
rng = numpy.random.RandomState(1234)

##  Step1. Define Symbolic / Shared Variables
x, t = T.fvector("x"), T.fvector("t") #inputs

W = theano.shared(rng.uniform(low=-0.08, high=0.08, size=(5, 3)), name="W") #variables that are shared over iterations
b = theano.shared(numpy.zeros(3), name="bias")


## Step2. Define graph
y = T.dot(x, W) + b
#y = T.nnet.sigmoid(T.dot(x, W) + b)
#y = T.tanh(T.dot(x, W) + b)
cost = T.sum((y - t)**2)


gW, gb = T.grad(cost, [W, b]) # Take gradient

updates =  OrderedDict({W: W-0.01*gW, b: b-0.01*gb}) # Set update expression in OrderedDict


## Step3. Compile graph
f = theano.function(inputs=[x, t], outputs=[cost, gW, gb], updates=updates, allow_input_downcast=True)

## Step4. Run!!
for epoch in range(5):
    cost, gW, gb = f([-2., -1., 1., 2., 3.], [.4, .3, .5])
    print("epoch:: %d, cost:: %.3f"%(epoch, cost))

**Since we have Autodiff, it's easy to change linear regression to non-linear regression.** Try it!!

## Multi Layer Perceptron (MLP) 

<img src="http://k-kawakami.com/img/mlp.png">

In [3]:
train_x, valid_x, train_y, valid_y = train_test_split(mnist_x, mnist_y, test_size=0.2, random_state=42)

In [4]:
#Multi Layer Perceptron
rng = numpy.random.RandomState(1234)

class Layer:
    def __init__(self, in_dim, out_dim, function):
        self.W = theano.shared(
            rng.uniform(low=-0.08, high=0.08, size=(in_dim, out_dim)).astype('float32'),
            name='W'
        )
        self.b = theano.shared(numpy.zeros(out_dim).astype('float32'), name='bias')
        self.func = function
        self.params = [self.W, self.b]
        
        # 前回のパラメータ更新量 (P.52)
        self.prev_dW = theano.shared(numpy.zeros((in_dim, out_dim)).astype('float32'), name='prev_dW')
        self.prev_db = theano.shared(numpy.zeros(out_dim).astype('float32'), name='prev_db')
        self.prev_dparams = [self.prev_dW, self.prev_db]

    def fprop(self, x):
        h = self.func(T.dot(x, self.W) + self.b)
        self.h = h # hは教科書のz
        return h
    
    def predict(self, x):
        return self.fprop(x)

In [5]:
def fprops(layers, x):
    '''
    ネットワーク全体の順伝播

    :param layers: ネットワーク 
    :param x: 入力
    :return: 出力層の出力
    '''
    z = x
    for layer in layers:
        z = layer.fprop(z)    
    return z

In [6]:
# Cost Function (Negative Log Likelihood)
def cross_entropy(y, d):
    '''
    交差エントロピーを計算する
    See: (2.11) and #3.3

    :param y: 出力層の出力
    :param d: 目標出力
    :return: 交差エントロピー
    '''
    # cf. http://deeplearning.net/tutorial/logreg.html#defining-a-loss-function
    # cf. http://docs.scipy.org/doc/numpy/reference/arrays.indexing.html#advanced-indexing
    return -T.mean(T.log(y)[T.arange(d.shape[0]), d])

In [7]:
def sgd(params, gparams, eps=0.01):
    '''
    stochastic gradient descent
    See: #3.2 and P.52
    
    :param params: 更新するパラメータ
    :param gparams: パラメータの勾配
    :param eps: 学習率
    :return: 更新後のパラメータが格納されたOrderedDict
    '''
    updates = OrderedDict()
    for param, gparam in zip(params, gparams):
        updates[param] = param - eps * gparam
    return updates

In [8]:
def sgd_with_momentum(params, gparams, prev_gparams, eps=0.01, mu=0.9):
    '''
    stochastic gradient descent with momentum
    See: #3.2, #3.6.4 and P.52
    
    :param params: 更新するパラメータ
    :param gparams: パラメータの勾配
    :param eps: 学習率
    :param mu: モメンタム
    :return: 更新後のパラメータが格納されたOrderedDict
    '''
    updates = OrderedDict()
    for param, gparam, prev_gparam in zip(params, gparams, prev_gparams):
        updates[param] = param - eps * gparam + mu * prev_gparam
        updates[prev_gparam] = - eps * gparam + mu * prev_gparam
    return updates

In [9]:
def predict(layers, x):
    '''
    学習済みネットワークを用いて推定する（ネットワーク全体の順伝播）

    :param layers: ネットワーク 
    :param x: 入力
    :return: 推定値（出力層の出力）
    '''
    z = x
    for layer in layers:
        z = layer.predict(z)    
    return z

In [10]:
x, t = T.fmatrix("x"), T.ivector("t")
activation = T.nnet.sigmoid #T.tanh

layers = [
    Layer(in_dim=784, out_dim=500, function=activation),
    Layer(in_dim=500, out_dim=500, function=activation),
    Layer(in_dim=500, out_dim=500, function=activation),
    Layer(in_dim=500, out_dim=10, function=T.nnet.softmax),
]

## Collect Parameters and Symbolic output
params = []
prev_dparams = []
for layer in layers:
    params += layer.params
    prev_dparams += layer.prev_dparams

y = fprops(layers, x)
cost = cross_entropy(y, t)

## Get Gradient
gparams = T.grad(cost, params)
#updates = sgd(params, gparams)
updates = sgd_with_momentum(params, gparams, prev_dparams)

## Compile 
train = theano.function([x,t], cost, updates=updates)
test = theano.function([x,t],[cost, T.argmax(y, axis=1)])

In [11]:
## Iterate
batch_size = 100
nbatches = train_x.shape[0] // batch_size

for epoch in range(500):
    train_x, train_y = shuffle(train_x, train_y)
    for i in range(nbatches):
        start = i * batch_size
        end = start + batch_size
        
        train(train_x[start:end], train_y[start:end])
    
    if ((epoch+1) % 10 == 0) or (epoch == 0):
        valid_cost, pred = test(valid_x, valid_y)
        print("EPOCH:: {:3d}, Validatioon Cost:: {:.3f}, Validation F1:: {:.3f}".format(epoch+1,
                                                                                     float(valid_cost),
                                                                                     f1_score(valid_y, pred, average="micro")))

EPOCH::   1, Validatioon Cost:: 2.174, Validation F1:: 0.223
EPOCH::  10, Validatioon Cost:: 0.332, Validation F1:: 0.904
EPOCH::  20, Validatioon Cost:: 0.230, Validation F1:: 0.934
EPOCH::  30, Validatioon Cost:: 0.165, Validation F1:: 0.953
EPOCH::  40, Validatioon Cost:: 0.132, Validation F1:: 0.961
EPOCH::  50, Validatioon Cost:: 0.116, Validation F1:: 0.966
EPOCH::  60, Validatioon Cost:: 0.104, Validation F1:: 0.970
EPOCH::  70, Validatioon Cost:: 0.102, Validation F1:: 0.969
EPOCH::  80, Validatioon Cost:: 0.099, Validation F1:: 0.974
EPOCH::  90, Validatioon Cost:: 0.092, Validation F1:: 0.975
EPOCH:: 100, Validatioon Cost:: 0.093, Validation F1:: 0.975
EPOCH:: 110, Validatioon Cost:: 0.094, Validation F1:: 0.976
EPOCH:: 120, Validatioon Cost:: 0.096, Validation F1:: 0.977
EPOCH:: 130, Validatioon Cost:: 0.098, Validation F1:: 0.976
EPOCH:: 140, Validatioon Cost:: 0.099, Validation F1:: 0.977
EPOCH:: 150, Validatioon Cost:: 0.101, Validation F1:: 0.977
EPOCH:: 160, Validatioon

### Dropout

In [12]:
from theano.tensor.shared_randomstreams import RandomStreams
trng = RandomStreams(1234)

class Dropout(Layer):
    def __init__(self, in_dim, out_dim, function, p):
        super().__init__(in_dim, out_dim, function)
        self.p = numpy.float32(p)

    def fprop(self, x, use_noise=True):
        mask = trng.binomial(size=x.shape, n=1, p=self.p).astype('float32') # int32だとtheano.functionでコンパイル出来ない:(
        h = self.func(T.dot(x*mask, self.W) + self.b) # TODO: check bias
        self.h = h
        return h
    
    def predict(self, x):
        # #3.5.3
        h = self.func(T.dot(x, self.p*self.W) + self.b)
        self.h = h
        return h

#### train

In [13]:
x, t = T.fmatrix("x"), T.ivector("t")
activation = T.nnet.sigmoid #T.tanh

layers = [
    Dropout(in_dim=784, out_dim=1000, function=activation, p=0.9),
    Dropout(in_dim=1000, out_dim=1000, function=activation, p=0.5),
    Dropout(in_dim=1000, out_dim=1000, function=activation, p=0.5),
    Layer(in_dim=1000, out_dim=10, function=T.nnet.softmax),
]

## Collect Parameters and Symbolic output
params = []
prev_dparams = []
for layer in layers:
    params += layer.params
    prev_dparams += layer.prev_dparams

y = fprops(layers, x)
cost = cross_entropy(y, t)

## Get Gradient
gparams = T.grad(cost, params)
#updates = sgd(params, gparams)
updates = sgd_with_momentum(params, gparams, prev_dparams)

## Compile 
train = theano.function([x,t], cost, updates=updates)

#### test

In [14]:
y = predict(layers, x)
cost = cross_entropy(y, t)

## Compile 
test = theano.function([x,t],[cost, T.argmax(y, axis=1)])

#### Iterate

In [16]:
batch_size = 100
nbatches = train_x.shape[0] // batch_size

for epoch in range(500):
    train_x, train_y = shuffle(train_x, train_y)
    for i in range(nbatches):
        start = i * batch_size
        end = start + batch_size
        
        train(train_x[start:end], train_y[start:end])
    
    if ((epoch+1) % 10 == 0) or (epoch == 0):
        valid_cost, pred = test(valid_x, valid_y)
        print("EPOCH:: {:3d}, Validatioon Cost:: {:.3f}, Validation F1:: {:.3f}".format(epoch+1,
                                                                                     float(valid_cost),
                                                                                     f1_score(valid_y, pred, average="micro")))

EPOCH::   1, Validatioon Cost:: 2.280, Validation F1:: 0.115
EPOCH::  10, Validatioon Cost:: 0.508, Validation F1:: 0.848
EPOCH::  20, Validatioon Cost:: 0.375, Validation F1:: 0.891
EPOCH::  30, Validatioon Cost:: 0.333, Validation F1:: 0.901
EPOCH::  40, Validatioon Cost:: 0.292, Validation F1:: 0.916
EPOCH::  50, Validatioon Cost:: 0.261, Validation F1:: 0.923
EPOCH::  60, Validatioon Cost:: 0.235, Validation F1:: 0.929
EPOCH::  70, Validatioon Cost:: 0.216, Validation F1:: 0.934
EPOCH::  80, Validatioon Cost:: 0.201, Validation F1:: 0.940
EPOCH::  90, Validatioon Cost:: 0.191, Validation F1:: 0.941
EPOCH:: 100, Validatioon Cost:: 0.173, Validation F1:: 0.947
EPOCH:: 110, Validatioon Cost:: 0.164, Validation F1:: 0.950
EPOCH:: 120, Validatioon Cost:: 0.156, Validation F1:: 0.952
EPOCH:: 130, Validatioon Cost:: 0.148, Validation F1:: 0.955
EPOCH:: 140, Validatioon Cost:: 0.141, Validation F1:: 0.956
EPOCH:: 150, Validatioon Cost:: 0.136, Validation F1:: 0.959
EPOCH:: 160, Validatioon

#### Exercises: Squeeze out the last few percent!!
1. Try other objectives, MSE, Cross Entropy. 
2. Try symmetric activation functions (tanh, softsign).
3. Implement ReLU (Hint. look at Math section).
4. Add Dropout (Hint. apply random mask to inputs in fprop).
5. Add learning rate adjuster (Hint. make learning rate as shared variable).

## References
1. [Theano Documentation](http://deeplearning.net/software/theano/introduction.html)
2. [Deep Learning Tutorial](http://deeplearning.net/tutorial/)
3. F. Bastien, P. Lamblin, R. Pascanu, J. Bergstra, I. Goodfellow, A. Bergeron, N. Bouchard, D. Warde-Farley and Y. Bengio. [Theano: new features nd speed improvements"](http://arxiv.org/pdf/1211.5590.pdf). NIPS 2012 deep learning workshop.
4. J. Bergstra, O. Breuleux, F. Bastien, P. Lamblin, R. Pascanu, G. Desjardins, J. Turian, D. Warde-Farley and Y. Bengio. [Theano: A CPU and GPU Math Expression Compiler](http://www.iro.umontreal.ca/~lisa/pointeurs/theano_scipy2010.pdf). Proceedings of the Python for Scientific Computing Conference (SciPy) 2010. June 30 - July 3, Austin, TX
5. [Wikipedia, Convolution](https://en.wikipedia.org/wiki/Convolution)
6. [Stanford UFLDL Tutorial](http://ufldl.stanford.edu/wiki/index.php/UFLDL_Tutorial)

## Acknowledgements

Thank you Guillaume Lample, Ramón Fernandez Astudillo, Chu-Cheng Lin for reviewing this tutorial. 

And I appreciate Theano developers!!!